In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pylab as pl
import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions

2023-02-08 20:00:33.490708: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 20:00:33.945582: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-08 20:00:33.945653: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-08 20:00:36.523193: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [4]:
df = pd.read_csv('../datasets/train.csv')
pd.set_option('display.max_columns', None)
df

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,14,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,3,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,3,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,5,336,670,869,18,10,19,1,1,1,0


In [5]:
df.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [6]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [7]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X = mms.fit_transform(X)

In [8]:
y = pd.get_dummies(y)
y

,0,1,2,3
0,0,1,0,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,1,0,0
...,...,...,...,...
1995,1,0,0,0
1996,0,0,1,0
1997,0,0,0,1
1998,1,0,0,0


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [10]:
y_train.shape

(1500, 4)

In [11]:
model = tf.keras.models.Sequential(
[
    tf.keras.layers.Dense(20 , activation='relu' , input_shape=(20,)),
    tf.keras.layers.Dense(100 , activation='relu'),
    tf.keras.layers.Dense(20 , activation='relu'),
    tf.keras.layers.Dense(4 , activation='softmax')    
])

2023-02-08 20:00:42.914411: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-08 20:00:42.914492: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-08 20:00:42.914529: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (younes-Lenovo-ideapad-130-15AST): /proc/driver/nvidia/version does not exist
2023-02-08 20:00:42.915014: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
mcp = model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='../models/mlp.h5',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [14]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=32 , callbacks=[mcp])

Epoch 1/100
47/47 [==============================] - 5s 33ms/step - loss: 1.3724 - accuracy: 0.2947 - val_loss: 1.3290 - val_accuracy: 0.4100
Epoch 2/100
47/47 [==============================] - 1s 16ms/step - loss: 1.2913 - accuracy: 0.4000 - val_loss: 1.1812 - val_accuracy: 0.5260
Epoch 3/100
47/47 [==============================] - 1s 17ms/step - loss: 1.0776 - accuracy: 0.5540 - val_loss: 0.9163 - val_accuracy: 0.6580
Epoch 4/100
47/47 [==============================] - 1s 13ms/step - loss: 0.7954 - accuracy: 0.7040 - val_loss: 0.6898 - val_accuracy: 0.7560
Epoch 5/100
47/47 [==============================] - 1s 17ms/step - loss: 0.6194 - accuracy: 0.7627 - val_loss: 0.5630 - val_accuracy: 0.7960
Epoch 6/100
47/47 [==============================] - 1s 15ms/step - loss: 0.5261 - accuracy: 0.8000 - val_loss: 0.4855 - val_accuracy: 0.8160
Epoch 7/100
47/47 [==============================] - 1s 14ms/step - loss: 0.4539 - accuracy: 0.8247 - val_loss: 0.4253 - val_accuracy: 0.8580
Epoch 

47/47 [==============================] - 1s 12ms/step - loss: 0.0741 - accuracy: 0.9747 - val_loss: 0.1257 - val_accuracy: 0.9380
Epoch 59/100
47/47 [==============================] - 1s 12ms/step - loss: 0.0835 - accuracy: 0.9700 - val_loss: 0.1185 - val_accuracy: 0.9460
Epoch 60/100
47/47 [==============================] - 1s 12ms/step - loss: 0.0722 - accuracy: 0.9740 - val_loss: 0.1333 - val_accuracy: 0.9360
Epoch 61/100
47/47 [==============================] - 0s 10ms/step - loss: 0.0665 - accuracy: 0.9820 - val_loss: 0.1321 - val_accuracy: 0.9460
Epoch 62/100
47/47 [==============================] - 1s 12ms/step - loss: 0.0843 - accuracy: 0.9687 - val_loss: 0.1330 - val_accuracy: 0.9360
Epoch 63/100
47/47 [==============================] - 1s 13ms/step - loss: 0.0869 - accuracy: 0.9667 - val_loss: 0.1313 - val_accuracy: 0.9320
Epoch 64/100
47/47 [==============================] - 1s 12ms/step - loss: 0.0685 - accuracy: 0.9813 - val_loss: 0.1470 - val_accuracy: 0.9240
Epoch 65/100